### Questions

### Outcomes
YWBAT
- perform a query that is ordered by a value
- load query results as a dataframe
- build functions to perform queries
- execute a query using multiple joins

### Outline
* Take Questions
* Warm Up
* Connect to our sqlite db
* Compare and contrast reading a query as a list vs reading a query as a dataframe
* Practice various queries
* Complete a Join Query
* Complete a query using multiple Joins
* wrap up

### (5 min) Warm Up
Given the following 'Employees' table, write a query that completes the following task.  Send your query to me in a private chat here in zoom.

Expected Result:
- IdNum, LName, FName and Salaryj
- Order it by salary, starting with the highest salary
![](images/table.png)

<details>
    <summary>Solution</summary>
    
    ```
    SELECT IdNum, LName, FName, Salary
    FROM Employees
    ORDER BY Salary DESC
    ```
</details>

In [ ]:
# Getting connected to 

In [1]:
import pandas as pd
import numpy as np

import sqlite3

import matplotlib.pyplot as plt

### Connecting to our sqlite db using sqlite3

In [2]:
conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor()

### Listing the tables in our db

In [3]:
# table_names to be a list of my table_names
query = "SELECT name FROM sqlite_master WHERE type='table';"
res = cursor.execute(query).fetchall()
print(res) # Notice these are tuples, let's extract only the names in the next cell

[('orderdetails',), ('payments',), ('offices',), ('customers',), ('orders',), ('productlines',), ('products',), ('employees',), ('contacts',), ('contacts2',)]


In [4]:
table_names = [r[0] for r in res]
table_names

['orderdetails',
 'payments',
 'offices',
 'customers',
 'orders',
 'productlines',
 'products',
 'employees',
 'contacts',
 'contacts2']

### Now  select everything from the employees table just to get a feel for it

In [7]:
query = 'select * from employees;'
res = cursor.execute(query).fetchall()
res[:2]

[('1002',
  'Murphy',
  'Diane',
  'x5800',
  'dmurphy@classicmodelcars.com',
  '1',
  '',
  'President'),
 ('1056',
  'Patterson',
  'Mary',
  'x4611',
  'mpatterso@classicmodelcars.com',
  '1',
  '1002',
  'VP Sales')]

### Now let's load this into a dataframe using the `.read_sql` method, we'll use some of the same components from above. 

In [9]:
query = 'select * from employees;'
df = pd.read_sql(query, conn)
df.head() # Much better and more readable! 

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)


### (3 min) What are the pros and cons of loading a sql query into a dataframe?
- Pros
    - 

- Cons
    -  

In [3]:
def load_df(table_name=None, conn=None):
    query = 'select * from {}'.format(table_name)
    df = pd.read_sql(query, conn)
    return df

### A throwback favorite 

In [6]:
d = {} # table_name: dataframe of table
for table_name in table_names:
    d[table_name] = load_df(table_name, conn)

### WHY SHOULD YOU ALWAYS SPECIFY YOUR PRIMARY KEYS!?

SQL will get super mad if we try and produce duplicates. It keeps us safe. 

In [42]:
customers_df.head(2)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.00


In [41]:
orders_df = pd.read_sql("select * from orders", conn)
orders_df.head()

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141


In [47]:
orders_df.shape, join_1.shape

((326, 7), (326, 4))

In [44]:
# let's do a join on customer number
# customerName, customer.phone,  customer state, customer order-date
query = """
        SELECT c.customerName, c.phone, c.state, o.orderDate FROM
        customers as c
        JOIN orders as o using (customerNumber);
        """

In [46]:
join_1 = pd.read_sql(query, conn)
join_1.head()

,customerName,phone,state,orderDate
0,Atelier graphique,40.32.2555,,2003-05-20
1,Atelier graphique,40.32.2555,,2004-09-27
2,Atelier graphique,40.32.2555,,2004-11-25
3,Signal Gift Stores,7025551838,NV,2003-05-21
4,Signal Gift Stores,7025551838,NV,2004-08-06


### Level Up: Display the names of each product each employee has sold

In [7]:
employees_df = pd.read_sql("select * from employees", conn)
employees_df.head(1)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President


In [8]:
customers_df.head(1)

NameError: name 'customers_df' is not defined

In [66]:
query1 = """
        create temporary table emp_cust as
        select e.lastName, e.firstName, c.salesRepEmployeeNumber, c.customerNumber
        from employees as e
        join customers as c 
        on e.employeeNumber = c.salesRepEmployeeNumber;
        """

query2 = """
         create temporary table eco as 
         select ec.customerNumber, ec.lastName, ec.firstName, o.orderNumber
         from emp_cust as ec
         join orders as o
         using (customerNumber);"""


# cursor.execute(query1).fetchall()
cursor.execute(query2).fetchall()

[]

In [73]:
query3 = """
         create temporary table ecod as
         select eco.orderNumber, eco.lastName, eco.firstName, eco.orderNumber, od.productCode
         from eco
         join orderdetails as od
         using (orderNumber);"""
# cursor.execute(query3).fetchall()

In [51]:
orders_df.head(1)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363


In [85]:
query = 'select * from eco;'
cursor.execute(query).fetchall()[:3]

[('124', 'Jennings', 'Leslie', '10113'),
 ('124', 'Jennings', 'Leslie', '10135'),
 ('124', 'Jennings', 'Leslie', '10142')]

In [52]:
orderdetails_df.head(1)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3


In [84]:
query = """
        select eco.firstName, eco.lastName, eco.productCode, p.productName
        from ecod as eco
        join products as p
        using (productCode)
        group by eco.firstName, eco.lastName, p.productName
        order by eco.firstName, p.productName"""
cursor.execute(query).fetchall()[:3]

[('Andy', 'Fixter', 'S18_3136', '18th Century Vintage Horse Carriage'),
 ('Andy', 'Fixter', 'S24_2841', '1900s Vintage Bi-Plane'),
 ('Andy', 'Fixter', 'S24_4278', '1900s Vintage Tri-Plane')]

In [53]:
products_df.head(1)

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70


### What did you learn?
* PRAGMA
* Making long strings
* Temporary tables
* pd.read_sql
* dictionary key:value -> tablename: dataframes

# Subquery Section

In [24]:
### let's look at employee customer tables first
employees_df.head(1)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President


In [26]:
customers_df = pd.read_sql("select * from customers;", conn)
customers_df.head(1)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00


# Query 0

In [30]:
query = """
        select e.firstname, e.lastname, e.employeenumber, c.customerNumber
        from employees as e
        join customers as c
        on e.employeenumber = c.salesrepemployeenumber;"""

pd.read_sql(query, conn).head(2)

,firstName,lastName,employeeNumber,customerNumber
0,Leslie,Jennings,1165,124
1,Leslie,Jennings,1165,129


In [29]:
# Now let's take this and incorporate the next table orders
orders_df.head(1)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363


# Query 1

In [39]:
# add orders to previous queries joining on customernumber
query = """
        select e.firstname, e.lastname, e.employeenumber, c.customerNumber, o.orderNumber
        from employees as e
        join customers as c
        on e.employeenumber = c.salesrepemployeenumber
        join orders as o
        on o.customerNumber = c.customerNumber;"""

pd.read_sql(query, conn).head(2)

,firstName,lastName,employeeNumber,customerNumber,orderNumber
0,Leslie,Jennings,1165,124,10113
1,Leslie,Jennings,1165,124,10135


In [40]:
# now let's incororate the orderdetails table
orderdetails_df.head(1)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3


# Query 2

In [42]:
# adding product code, removing employee number and customer number
query = """
        select e.firstname, e.lastname, o.orderNumber, od.productCode
        from employees as e
        join customers as c
        on e.employeenumber = c.salesrepemployeenumber
        join orders as o
        on o.customerNumber = c.customerNumber
        join orderdetails as od
        on od.orderNumber = o.orderNumber;"""

pd.read_sql(query, conn).head(2)

,firstName,lastName,orderNumber,productCode
0,Leslie,Jennings,10113,S12_1666
1,Leslie,Jennings,10113,S18_1097


In [43]:
# now let's investigate the product table
pd.read_sql("select * from products", conn).head(1)

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70


# Query 3 - Final

In [45]:
# Now let's add product name with some final group by to remove duplicates
# removing order number and product code
query = """
        select e.firstname, e.lastname, p.productName
        from employees as e
        join customers as c
        on e.employeenumber = c.salesrepemployeenumber
        join orders as o
        on o.customerNumber = c.customerNumber
        join orderdetails as od
        on od.orderNumber = o.orderNumber
        join products as p
        on p.productCode = od.productCode;"""

pd.read_sql(query, conn).head(2)

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck


### Great! It's working. Let's add some group bys for duplicates!

In [52]:
# alias the columns and add group by at the bottom
query = """
        select e.firstname as fn, e.lastname as ln, p.productName as pn
        from employees as e
        join customers as c
        on e.employeenumber = c.salesrepemployeenumber
        join orders as o
        on o.customerNumber = c.customerNumber
        join orderdetails as od
        on od.orderNumber = o.orderNumber
        join products as p
        on p.productCode = od.productCode
        group by fn, ln, pn;"""

final_df = pd.read_sql(query, conn)
final_df.head(2)

,fn,ln,pn
0,Andy,Fixter,18th Century Vintage Horse Carriage
1,Andy,Fixter,1900s Vintage Bi-Plane


In [57]:
final_df.shape, final_df.drop_duplicates().shape

((1368, 3), (1368, 3))